In [6]:
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from typing import TypedDict, Literal, Annotated
from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI
from pydantic import BaseModel, Field
from langchain_core.messages import SystemMessage, HumanMessage, BaseMessage
from langgraph.checkpoint.memory import InMemorySaver

class JokeState(TypedDict):
    topic: str
    joke: str
    explanation: str

load_dotenv()
llm=AzureChatOpenAI(model="gpt-4o-mini")  
    
def generate_joke(state: JokeState):
    prompt=f'Generate a joke about {state["topic"]}'
    response=llm.invoke(prompt).content
    return {'joke': response}

def generate_explanation(state: JokeState):
    prompt=f'Explain the following joke in simple terms: {state["joke"]}'
    response=llm.invoke(prompt).content
    return {'explanation': response}
 
 
graph = StateGraph(JokeState)

#add nodes
graph.add_node('generate_joke', generate_joke)
graph.add_node('generate_explanation', generate_explanation)

#add edges
graph.add_edge(START, 'generate_joke')
graph.add_edge('generate_joke', 'generate_explanation')
graph.add_edge('generate_explanation', END)


checkpointer = InMemorySaver()  


workflow = graph.compile(checkpointer=checkpointer)

thread_id = '1'
# Initialize only once checkpointer for the thread
config = {"configurable": {"thread_id": thread_id}}

output=workflow.invoke({"topic": "programming"}, config=config)
print(output)

{'topic': 'programming', 'joke': 'Why do programmers prefer dark mode? \n\nBecause light attracts bugs!', 'explanation': 'This joke plays on two meanings of the word "bugs." \n\n1. In programming, "bugs" refer to errors or problems in code that can cause the software to work incorrectly.\n2. In the real world, lights attract insects, which are commonly referred to as "bugs."\n\nSo, the joke suggests that programmers prefer dark mode (a screen setting with a dark background) because it prevents attracting those pesky "bugs" (both in the form of insects and coding errors). It\'s a clever play on words that connects coding culture to a familiar phenomenon in nature.'}


In [7]:
list(workflow.get_state(config))

[{'topic': 'programming',
  'joke': 'Why do programmers prefer dark mode? \n\nBecause light attracts bugs!',
  'explanation': 'This joke plays on two meanings of the word "bugs." \n\n1. In programming, "bugs" refer to errors or problems in code that can cause the software to work incorrectly.\n2. In the real world, lights attract insects, which are commonly referred to as "bugs."\n\nSo, the joke suggests that programmers prefer dark mode (a screen setting with a dark background) because it prevents attracting those pesky "bugs" (both in the form of insects and coding errors). It\'s a clever play on words that connects coding culture to a familiar phenomenon in nature.'},
 (),
 {'configurable': {'thread_id': '1',
   'checkpoint_ns': '',
   'checkpoint_id': '1f0c362a-2df6-6c2f-8002-b7ba0665500c'}},
 {'source': 'loop', 'step': 2, 'parents': {}},
 '2025-11-17T03:08:05.524750+00:00',
 {'configurable': {'thread_id': '1',
   'checkpoint_ns': '',
   'checkpoint_id': '1f0c362a-2259-6b28-8001-dd

In [8]:
list(workflow.get_state_history(config))

[StateSnapshot(values={'topic': 'programming', 'joke': 'Why do programmers prefer dark mode? \n\nBecause light attracts bugs!', 'explanation': 'This joke plays on two meanings of the word "bugs." \n\n1. In programming, "bugs" refer to errors or problems in code that can cause the software to work incorrectly.\n2. In the real world, lights attract insects, which are commonly referred to as "bugs."\n\nSo, the joke suggests that programmers prefer dark mode (a screen setting with a dark background) because it prevents attracting those pesky "bugs" (both in the form of insects and coding errors). It\'s a clever play on words that connects coding culture to a familiar phenomenon in nature.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0c362a-2df6-6c2f-8002-b7ba0665500c'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-11-17T03:08:05.524750+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': ''

In [9]:
workflow = graph.compile(checkpointer=checkpointer)
thread_id = '2'
# Initialize only once checkpointer for the thread
config1 = {"configurable": {"thread_id": thread_id}}

workflow.invoke({"topic": "pizza"}, config=config1)

{'topic': 'pizza',
 'joke': "Why did the pizza maker go broke? \n\nBecause he just couldn't make enough dough! 🍕",
 'explanation': 'The joke is a play on words. "Dough" has two meanings: \n\n1. It refers to the bread mixture that pizza makers use to make pizzas.\n2. It is also a slang term for money.\n\nSo, when the joke says the pizza maker "just couldn\'t make enough dough," it means he couldn\'t make enough pizza dough and also implies that he couldn\'t make enough money. The humor comes from the double meaning of "dough."'}

In [12]:
list(workflow.get_state_history(config))

[StateSnapshot(values={'topic': 'programming', 'joke': 'Why do programmers prefer dark mode? \n\nBecause light attracts bugs!', 'explanation': 'This joke plays on two meanings of the word "bugs." \n\n1. In programming, "bugs" refer to errors or problems in code that can cause the software to work incorrectly.\n2. In the real world, lights attract insects, which are commonly referred to as "bugs."\n\nSo, the joke suggests that programmers prefer dark mode (a screen setting with a dark background) because it prevents attracting those pesky "bugs" (both in the form of insects and coding errors). It\'s a clever play on words that connects coding culture to a familiar phenomenon in nature.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0c362a-2df6-6c2f-8002-b7ba0665500c'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-11-17T03:08:05.524750+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': ''

In [13]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': "Why did the pizza maker go broke? \n\nBecause he just couldn't make enough dough! 🍕", 'explanation': 'The joke is a play on words. "Dough" has two meanings: \n\n1. It refers to the bread mixture that pizza makers use to make pizzas.\n2. It is also a slang term for money.\n\nSo, when the joke says the pizza maker "just couldn\'t make enough dough," it means he couldn\'t make enough pizza dough and also implies that he couldn\'t make enough money. The humor comes from the double meaning of "dough."'}, next=(), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f0c362a-5afd-6f12-8002-00bdde76bd59'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-11-17T03:08:10.246288+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f0c362a-4fcc-6fe7-8001-760297ed40ed'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'topic': 'pizza', 'joke': "Why d

# Time Travel

In [19]:
workflow.get_state({"configurable": {"thread_id": '2', "checkpoint_id":"1f0c362a-4939-6219-8000-205e6a8ea377"}})

StateSnapshot(values={'topic': 'pizza'}, next=('generate_joke',), config={'configurable': {'thread_id': '2', 'checkpoint_id': '1f0c362a-4939-6219-8000-205e6a8ea377'}}, metadata={'source': 'loop', 'step': 0, 'parents': {}}, created_at='2025-11-17T03:08:08.383106+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f0c362a-4930-61c9-bfff-69c9f90ccc81'}}, tasks=(PregelTask(id='17abfa6d-23af-d2d9-81bf-9cf29847b9d7', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result={'joke': "Why did the pizza maker go broke? \n\nBecause he just couldn't make enough dough! 🍕"}),), interrupts=())

In [20]:
workflow.invoke(None, config={"configurable": {"thread_id": '2', "checkpoint_id":"1f0c362a-4939-6219-8000-205e6a8ea377"}})

{'topic': 'pizza',
 'joke': 'Why did the pizza apply for a job?\n\nBecause it kneaded the dough! 🍕😄',
 'explanation': 'This joke plays with the word "dough." In the context of pizza, "dough" is the mixture used to make the base of the pizza. However, "dough" is also a slang term for money. \n\nSo when it says the pizza "kneaded the dough," it’s a pun: "kneaded" sounds like "needed." The joke is that the pizza is looking for a job because it "needs" money. The humor comes from imagining a pizza wanting to work and making a play on words with "knead" (like working with dough) and "need" (like needing money).'}

In [21]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza apply for a job?\n\nBecause it kneaded the dough! 🍕😄', 'explanation': 'This joke plays with the word "dough." In the context of pizza, "dough" is the mixture used to make the base of the pizza. However, "dough" is also a slang term for money. \n\nSo when it says the pizza "kneaded the dough," it’s a pun: "kneaded" sounds like "needed." The joke is that the pizza is looking for a job because it "needs" money. The humor comes from imagining a pizza wanting to work and making a play on words with "knead" (like working with dough) and "need" (like needing money).'}, next=(), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f0c365a-8e1c-6418-8002-230b51ffbc45'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-11-17T03:29:44.096625+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f0c365a-7bb4-68ef-8001-a2f9cb63bcf8'}},

# Updating State

In [22]:
workflow.update_state({"configurable": {"thread_id": '2', "checkpoint_id":"1f0c362a-4939-6219-8000-205e6a8ea377", "checkpoint_ns":""}}, {"topic":"samosas"})

{'configurable': {'thread_id': '2',
  'checkpoint_ns': '',
  'checkpoint_id': '1f0c3664-55fa-6440-8001-18542ed6196a'}}

In [24]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'samosas'}, next=('generate_joke',), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f0c3664-55fa-6440-8001-18542ed6196a'}}, metadata={'source': 'update', 'step': 1, 'parents': {}}, created_at='2025-11-17T03:34:06.646144+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f0c362a-4939-6219-8000-205e6a8ea377'}}, tasks=(PregelTask(id='b6c16b40-260d-2ca9-f091-cbff29ffbc07', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza apply for a job?\n\nBecause it kneaded the dough! 🍕😄', 'explanation': 'This joke plays with the word "dough." In the context of pizza, "dough" is the mixture used to make the base of the pizza. However, "dough" is also a slang term for money. \n\nSo when it says the pizza "kneaded the dough," it’s a pun: "knea

In [26]:
workflow.invoke(None, config={"configurable": {"thread_id": '2', "checkpoint_id":"1f0c3664-55fa-6440-8001-18542ed6196a"}})

{'topic': 'samosas',
 'joke': 'Why did the samosa go to school?\n\nBecause it wanted to be a little "more filling"!',
 'explanation': 'The joke is a play on words. A samosa is a type of stuffed pastry that is often filled with delicious ingredients, making it "filling" when you eat it. The joke implies that the samosa wants to go to school to learn and gain more knowledge, which would make it "more fulfilling" in a different sense—meaning more complete or satisfying in life. The pun comes from the similarity in sound between "filling" (what you eat) and "fulfilling" (having a purpose or being satisfying). So, it\'s funny because it combines these two meanings in a clever way.'}

In [27]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'samosas', 'joke': 'Why did the samosa go to school?\n\nBecause it wanted to be a little "more filling"!', 'explanation': 'The joke is a play on words. A samosa is a type of stuffed pastry that is often filled with delicious ingredients, making it "filling" when you eat it. The joke implies that the samosa wants to go to school to learn and gain more knowledge, which would make it "more fulfilling" in a different sense—meaning more complete or satisfying in life. The pun comes from the similarity in sound between "filling" (what you eat) and "fulfilling" (having a purpose or being satisfying). So, it\'s funny because it combines these two meanings in a clever way.'}, next=(), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f0c3669-6dd4-6913-8003-f2a99d6afb2e'}}, metadata={'source': 'loop', 'step': 3, 'parents': {}}, created_at='2025-11-17T03:36:23.365018+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpo